In [1]:
import json
import yaml
import numpy as np
import shutil
from pathlib import Path
import pandas as pd
from mouselab.mouselab import MouselabEnv
from mouselab.env_utils import get_ground_truths_from_json

In [2]:
discount_factors = [.50,.75,.85,.9,.95,.975,.999]

In [3]:
for discount_factor in discount_factors:
    shutil.copyfile( Path(f"../../../data/inputs/exp_inputs/structure/312_2_4_24.json"), \
                     Path(f"../../../data/inputs/exp_inputs/structure/high_increasing{discount_factor}.json"))

In [4]:
with open(Path("../../../data/inputs/yamls/experiment_settings/high_increasing.yaml"), "rb") as f:
    experiment_setting_data = yaml.safe_load(f)
    
for discount_factor in discount_factors:
    experiment_setting_data["ground_truth_file"] = f"high_increasing{discount_factor}"
    experiment_setting_data["structure"] = f"high_increasing{discount_factor}"
    
    with open(Path(f"../../../data/inputs/yamls/experiment_settings/high_increasing{discount_factor}.yaml"), "w") as f:
        yaml.dump(experiment_setting_data, f)

In [5]:
env = MouselabEnv.new_symmetric_registered("high_increasing")

for discount_factor in discount_factors:
    with open(Path("../../../data/inputs/exp_inputs/rewards/312_2_4_24.json"), "rb") as f:
        data = json.load(f)
    
    for trial in data:
        trial["stateRewards"] = [round(reward*discount_factor**(env.mdp_graph.nodes[action]["depth"]-1),2) if env.mdp_graph.nodes[action]["depth"] != 0 else 0.0 for action, reward in enumerate(trial["stateRewards"])]

    with open(Path(f"../../../data/inputs/exp_inputs/rewards/high_increasing{discount_factor}.json"), "w") as f:
        json.dump(data, f)

In [6]:
ls ../../../data/inputs/yamls/experiments

c1.1.yaml              methods_main0.95.yaml        quest_feasibility.yaml
c2.1.yaml              methods_main0.975.yaml       quest_first.yaml
irl_validation.yaml    methods_main0.999.yaml       quest_main.yaml
methods_main0.5.yaml   methods_main0.9.yaml         quest_second.yaml
methods_main0.75.yaml  methods_main_extension.yaml
methods_main0.85.yaml  methods_main.yaml


In [7]:
with open(Path("../../../data/inputs/yamls/experiments/methods_main.yaml"), "rb") as f:
    data = yaml.safe_load(f)

for discount_factor in discount_factors:
    data['sessions'] = [f'methods_main{discount_factor}']
    data['experiment_setting'] = f'high_increasing{discount_factor}'
    with open(Path(f"../../../data/inputs/yamls/experiments/methods_main{discount_factor}.yaml"), "w") as f:
        yaml.dump(data, f)

Put this in `cluster/src/cluster_utils.py`

In [8]:
for discount_factor in discount_factors:
    print(f"elif name == 'high_increasing{discount_factor}':\n"
          f"    register(\n"
          f"        name=name,\n"
          f"        branching=[3, 1, 2],\n"
          f"        reward_inputs=\"depth\",\n"
          f"        reward_dictionary={{\n"
          f"            1: Categorical([-4, -2, 2, 4]),\n"
          f"            2: Categorical([{', '.join(str(round(val*discount_factor,2)) for val in [-8, -4, 4, 8])}]),\n"
          f"            3: Categorical([{', '.join(str(round(val*discount_factor**2,2)) for val in [-48, -24, 24, 48])}]),\n"
          f"        }},\n"
          f"    )")

elif name == 'high_increasing0.5':
    register(
        name=name,
        branching=[3, 1, 2],
        reward_inputs="depth",
        reward_dictionary={
            1: Categorical([-4, -2, 2, 4]),
            2: Categorical([-4.0, -2.0, 2.0, 4.0]),
            3: Categorical([-12.0, -6.0, 6.0, 12.0]),
        },
    )
elif name == 'high_increasing0.75':
    register(
        name=name,
        branching=[3, 1, 2],
        reward_inputs="depth",
        reward_dictionary={
            1: Categorical([-4, -2, 2, 4]),
            2: Categorical([-6.0, -3.0, 3.0, 6.0]),
            3: Categorical([-27.0, -13.5, 13.5, 27.0]),
        },
    )
elif name == 'high_increasing0.85':
    register(
        name=name,
        branching=[3, 1, 2],
        reward_inputs="depth",
        reward_dictionary={
            1: Categorical([-4, -2, 2, 4]),
            2: Categorical([-6.8, -3.4, 3.4, 6.8]),
            3: Categorical([-34.68, -17.34, 17.34, 34.68]),
        },
    )
elif name == 'high_in

In [9]:
np.savetxt(
    Path("../../../cluster/parameters/cost/null.txt"),
    [[0.0, 1.0]],
    fmt="%.02f",
    delimiter=",",
)

In [10]:
mouselab_data = pd.read_csv(Path("../../../data/processed/methods_main/mouselab-mdp.csv"), index_col=0)
for discount_factor in discount_factors:
    with open(Path(f"../../../data/inputs/exp_inputs/rewards/high_increasing{discount_factor}.json"), "rb") as f:
        data = json.load(f)
    ground_truth_dict = {trial["trial_id"] : trial["stateRewards"] for trial in data}

    mouselab_data["state_rewards"] = mouselab_data["trial_id"].apply(lambda trial_id : ground_truth_dict[trial_id][1:])
    
    # need to change ground truths
    Path(f"../../../data/processed/methods_main{discount_factor}").mkdir(exist_ok = True, parents=True)
    mouselab_data.to_csv(Path(f"../../../data/processed/methods_main{discount_factor}/mouselab-mdp.csv"), index=False)

Generate cluster submission commands to run to get Q values

In [11]:
for discount_factor in discount_factors:
    print(f"condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing{discount_factor} cost_function=linear_depth param_file=null")

condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing0.5 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing0.75 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing0.85 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing0.9 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing0.95 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing0.975 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/03_Get_Q_Values.sub experiment_setting=high_increasing0.999 cost_fu

Generate cluster submission commands to infer likelihood for methods_main:

In [12]:
for discount_factor in discount_factors:
    print(f"condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main{discount_factor} cost_function=linear_depth param_file=null")

condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main0.5 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main0.75 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main0.85 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main0.9 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main0.95 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main0.975 cost_function=linear_depth param_file=null
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main0.999 cost_function=linear_depth param_file=null
